In [23]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('../data/raw/kaggle_fake_news/train.csv')
test = pd.read_csv('../data/raw/kaggle_fake_news/test.csv')

In [4]:
train = train.dropna()
X_train, X_val, y_train, y_val = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [6]:
X_train_encoded = tokenizer(
    list(X_train.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_val_encoded = tokenizer(
    list(X_val.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [7]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [24]:
train_dataset = FakeNewsDataset(X_train_encoded, y_train.values)
val_dataset = FakeNewsDataset(X_val_encoded, y_val.values)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

## Training

**TODO**: How to add metric calculation during evaluation in trainer (compute_metric()) : https://huggingface.co/course/chapter3/3?fw=pt

In [9]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification

In [9]:
training_args = TrainingArguments(
    output_dir='../models',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [10]:
trainer.train()

c:\Users\msi\anaconda3\envs\nlp_project\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14628
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5487
  Number of trainable parameters = 66955010
  0%|          | 0/5487 [00:00<?, ?it/s]C:\Users\msi\AppData\Local\Temp\ipykernel_18456\2842400840.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.6964, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  0%|          | 20/5487 [00:09<27:02,  3.37it/s]

{'loss': 0.7107, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  1%|          | 30/5487 [00:12<28:05,  3.24it/s]

...

100%|█████████▉| 5480/5487 [36:51<00:02,  2.36it/s]

{'loss': 0.0001, 'learning_rate': 7.018247443352717e-08, 'epoch': 3.0}


100%|██████████| 5487/5487 [36:54<00:00,  2.77it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 5487/5487 [36:54<00:00,  2.48it/s]

{'train_runtime': 2214.7787, 'train_samples_per_second': 19.814, 'train_steps_per_second': 2.477, 'train_loss': 0.04305735688649524, 'epoch': 3.0}


TrainOutput(global_step=5487, training_loss=0.04305735688649524, metrics={'train_runtime': 2214.7787, 'train_samples_per_second': 19.814, 'train_steps_per_second': 2.477, 'train_loss': 0.04305735688649524, 'epoch': 3.0})

## Load best model

In [12]:
model = DistilBertForSequenceClassification.from_pretrained('../models/checkpoint-5000/')

In [47]:
model = model.to('cuda')

### Calculate performance

In [49]:
# calculate accuracy
from tqdm import tqdm
acc = 0.0
with torch.no_grad():
    for data in tqdm(val_loader):
        input_ids, labels = data['input_ids'].to('cuda'), data['labels'].to('cuda')
        out = torch.softmax(model(input_ids).logits, dim=1)
        acc += torch.sum(torch.argmax(out, dim=1) == labels) / len(input_ids)

  0%|          | 0/115 [00:00<?, ?it/s]C:\Users\msi\AppData\Local\Temp\ipykernel_16204\2842400840.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 115/115 [00:53<00:00,  2.17it/s]


In [52]:
print(f"Accuracy: {acc / len(val_loader)}")

Accuracy: 0.9978260397911072
